In [ ]:
import pandas as pd
import numpy as np

def group_timestamps(timestamps, avg_window_hours=24):
    # a list of timestamp ranges that are close to each other
    timestamp_groups = []

    # find timestamps that are within 24 hours of any other timestamp
    for i in timestamps:
        similar_timestamps = []
        for t in timestamps:
            if abs(t-i) < pd.Timedelta(hours=avg_window_hours):
                similar_timestamps.append(t)

        # remove the timestamps that are similar to the current timestamp, as they are already in the list
        for s in similar_timestamps:
            timestamps = timestamps[timestamps != s]

        if len(similar_timestamps) > 0:
            timestamp_groups.append(similar_timestamps)

    return timestamp_groups

with open('Lab Results Compiled.xlsx', 'rb') as f:
    input_df = pd.read_excel(f, sheet_name='Analytics Model - LIMS')

dashboard_data = []
output_df = pd.DataFrame()

# remove pesky trailing spaces
input_df['Tank'] = input_df['Tank'].str.rstrip()

tanks = input_df['Tank'].unique()

for t in tanks:
    # extract data relevant to current tank
    tank_df = input_df[input_df['Tank'] == t]

    timestamps = tank_df["SampleResults[Sampled Timestamp]"]

    # figure out if any timestamps are close enough to be averaged
    timestamp_groups = group_timestamps(timestamps, avg_window_hours=24)

    for g in timestamp_groups:
        # extract the data for the current timestamp group
        data_to_avg = tank_df[tank_df['SampleResults[Sampled Timestamp]'].isin(g)]

        determinands = data_to_avg['Determinand[Determinand Name]'].unique()

        for d in determinands:
            # extract the data for the current determinand
            determinand_df = data_to_avg[data_to_avg['Determinand[Determinand Name]'] == d]
            determinand_df.reset_index(drop=True, inplace=True)

            result = determinand_df['SampleResults[Sample Result]']
            unit = determinand_df['Determinand[Unit of Measure]'][0]
            sample_nums = determinand_df['SampleResults[SampleNumber]'].unique()

            # remove any < or > from the result
            result = result.astype(str).str.replace("<", "")
            result = result.astype(str).str.replace(">", "")
            result = pd.to_numeric(result, errors='raise')

            std_dev = result.std()
            std_dev = round(std_dev, 2)
            result = result.mean()
            result = round(result, 2)

            time = pd.Timestamp(g[0])

            # Some formatting for output

            # remove spaces from determinand
            determinand = d.replace(" ", "-")

            # add ST to tank name unless it is INLET
            tank = t
            # if tank != "INLET":
            #     tank = f"ST{tank}"

            if len(tank) == 5:
                tank = tank.replace(" ", ".")
            if len(tank) == 4:
                tank = tank.replace(" ", "C.")

            dashboard_sample = {
                "key"    : f"{tank}.{determinand}",
                "value"  : result,
                "epoch"  : time.timestamp()
            }

            df_sample = pd.DataFrame({
                "tank"        : tank,
                "determinand" : d,
                "value"       : result,
                "std_dev"     : std_dev,
                "unit"        : unit,
                "timestamp"   : time,
                "sample_nums" : str(sample_nums)
            }, index=[0])

            dashboard_data.append(dashboard_sample)
            output_df = pd.concat([output_df, df_sample], axis=0, ignore_index=True)
print(dashboard_data)
output_df
output_df.to_csv("lab_data.csv", index=False)



In [ ]:
input_df['Tank'].unique()
# input_df['Determinand[Determinand Name]'].unique()

In [ ]:
with pd.ExcelWriter("output.xlsx") as writer:

    output_df.to_excel(writer, sheet_name="Averaged Data")
    input_df.to_excel(writer, sheet_name="Original Data")
    

    tanks = ['CON', 'INS', '20C', '30C', 'INLET']
    for t in tanks:

        # select from output_df where tank name starts with e.g. "STINS"
        tab_df = output_df[output_df['tank'].str.startswith(t)]

        # arrange so the individual tanks are shown are side by side
        tab_df = tab_df.pivot_table(index=['determinand', 'unit' ], columns=['tank', 'timestamp'], values='value')
        tab_df.to_excel(writer, sheet_name=t)

In [ ]:


# # resize the columns in the excel file to fit the data, while handling merged cells

import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

wb = openpyxl.load_workbook("output.xlsx")

for ws in wb.worksheets:
    for col in ws.columns:
        max_length = 0
        try:
            column = col[0].column_letter
        except:
            column = col[1].column_letter



        for cell in col:
            cell.alignment = Alignment(wrap_text=True)
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))

        adjusted_width = (max_length + 2) * 1.2
        ws.column_dimensions[column].width = adjusted_width

# Tweak the format of the tank specific sheets
tanks = ['CON', 'INS', '20C', '30C', 'INLET']

for t in tanks:
    ws = wb[t]

    # set the format of row 2, columns c onwards to be a date
    for col in ws.columns:
        letter = col[1].column_letter
        if letter in ["A", "B"]:
            continue
        cell = col[1]
        cell.number_format = "mmm-dd"
        ws.column_dimensions[letter].width = 9


wb.save("output.xlsx")




In [ ]:
# create a new df from lab_df selecting columns "unit" "and determinand"
unit_mapping_df = lab_df[["unit", "determinand"]].drop_duplicates()

# make a new column that is the concatenation of the unit and determinand
unit_mapping_df["label"] = unit_mapping_df["determinand"] + " (" + unit_mapping_df["unit"] +")"
unit_mapping_dict = unit_mapping_df.set_index("determinand").to_dict()["label"]
unit_mapping_dict

In [ ]:
# import initial_state_backup.csv to dataframe
import pandas as pd

# Split into 2 to stay below github 100MB limit
sensor_df = pd.read_csv("initial_state_backup.csv", names=["timestamp", "key", "value"], low_memory=False)
sensor_df2 = pd.read_csv("initial_state_backup_august_onward.csv", names=["timestamp", "key", "value"], low_memory=False)
# sensor_df = pd.read_csv("initial_state_backup_2023-11-29.csv", names=["timestamp", "key", "value"], low_memory=False)


unit_mapping = {
    "ts" : "C",
    "tl" : "C",
    "ph" : "pH",
    "pr" : "mBar",
    "gc" : "%CH4",
}

labels_mapping = {
    "ts"                            : "Top Temp (C)",
    "tl"                            : "Bottom Temp (C)",
    "ph"                            : "pH",
    "pr"                            : "Pressure (mBar)",
    "gc"                            : "CH4 Concentration (%)",
    'Ammonia'                       : 'Ammonia (mg/l as N)',
    'Nitrate'                       : 'Nitrate (mg/l as N)',
    'Nitrite'                       : 'Nitrite (mg/l as N)',
    'Phosphorus'                    : 'Phosphorus (mgP/l)',
    'Soluble Chemical Oxygen Demand': 'Soluble Chemical Oxygen Demand (mgO₂/l)',
    'Sulphate'                      : 'Sulphate (mgSO₄/l)',
    'Suspended solids'              : 'Suspended solids (mg/l)',
    'Total Chemical Oxygen Demand'  : 'Total Chemical Oxygen Demand (mgO₂/l)',
    'Total Nitrogen (as N)'         : 'Total Nitrogen (as N) (mg/l as N)',
    'Total oxidised nitrogen'       : 'Total oxidised nitrogen (mg/l as N)',
    'Alkalinity'                    : 'Alkalinity (mgCaCO₃/l)',
    'Biochemical oxygen demand'     : 'Biochemical oxygen demand (mgO₂/l)',
    'Conductivity'                  : 'Conductivity (mS/cm)',
    'Manganese'                     : 'Manganese (mgMn/l)',
    'Total organic carbon'          : 'Total organic carbon (mgC/l)',
}

color_mapping = {
    "30C.1": "#c586c0",
    "30C.2": "#c586c0",
    "30C.3": "#c586c0",
    "20C.1": "#ce9178",
    "20C.2": "#ce9178",
    "20C.3": "#ce9178",
    "INS.1": "#dcdcaa",
    "INS.2": "#dcdcaa",
    "INS.3": "#dcdcaa",
    "CON.1": "#9cdcfe",
    "CON.2": "#9cdcfe",
    "CON.3": "#9cdcfe",
    "AMB"  : "#5edf7e",
    # "INLET": "#1e1e1e",
    "INLET": "#8e3030",
}

def clean_sensor_df(sensor_df):

    #remove keys we don't need
    sensor_df = sensor_df[~sensor_df["key"].str.contains(".tc")]
    sensor_df = sensor_df[~sensor_df["key"].str.contains("TEST")]
    sensor_df = sensor_df[~sensor_df["key"].str.contains("debug")]
    sensor_df = sensor_df[~sensor_df["key"].str.contains("feedcontrol")]

    # Reformat the key (Move the final number from the key to the end of the tank name, and drop "ST")
    sensor_df["key"] = sensor_df["key"].str[2:5]+"."+sensor_df["key"].str[-1]+sensor_df["key"].str[5:-1]

    # rename key CON.4.ts to AMB
    sensor_df["key"] = sensor_df["key"].str.replace("CON.4.ts","AMB.ts", regex=False)

    sensor_df["key"] = sensor_df["key"].str.replace("CON.4.gc","AMB.gc", regex=False)
    sensor_df["key"] = sensor_df["key"].str.replace("CON.4.pr","AMB.pr", regex=False)

    sensor_df = sensor_df[~sensor_df["key"].str.contains(".4.")]

    # add a new "determanind" column containing the last part of the key
    sensor_df["determinand"] = sensor_df["key"].str.split(".").str[-1]

    # add a new "tank" column containing the first part of the key
    sensor_df["tank"] = sensor_df["key"].str[:-3]

    # add a new "unit" column containing the unit for the determinand
    sensor_df["unit"] = sensor_df["determinand"].map(unit_mapping)

    # drop the key column
    sensor_df = sensor_df.drop(columns=["key"])

    # convert any values that are "*" to NaN
    sensor_df["value"] = sensor_df["value"].replace("*", np.nan)

    # convert the value column to numeric
    sensor_df["value"] = pd.to_numeric(sensor_df["value"], errors='raise')

    # delete any temperature measurements between 14th an 19th july 2023 which are less than 0 degrees
    sensor_df = sensor_df[~((sensor_df["timestamp"] > "2023-07-14") & (sensor_df["timestamp"] < "2023-07-19") & (sensor_df["determinand"] == "ts") & (sensor_df["value"] < 0))]

    return sensor_df

sensor_df = clean_sensor_df(sensor_df)
sensor_df2 = clean_sensor_df(sensor_df2)

sensor_df2.to_csv('sensor_data_cleaned_august_onward.csv', index=False)
sensor_df.to_csv('sensor_data_cleaned.csv', index=False)


# merge the two dataframes
sensor_df = pd.concat([sensor_df2, sensor_df], axis=0, ignore_index=True)
sensor_df

In [ ]:
# Create a new "lab_df" from output_df, but with only the relevant columns
lab_df = output_df[["timestamp", "value", "tank", "determinand", "unit"]].copy()

# join the sensor_df and lab_df dataframes
full_df = pd.concat([sensor_df, lab_df], axis=0, ignore_index=True)
# full_df = sensor_df
# full_df = lab_df
full_df

In [ ]:
import plotly.express as px
import plotly.io as pio

# pio.templates.default = "ggplot2"
pio.templates.default = "plotly_dark"


fig = px.line(full_df,
            x="timestamp",
            y="value",
            title='Gauldry Sensor Data',
            color="tank",
            custom_data=["value"],
            color_discrete_map=color_mapping,
            facet_row="determinand",
            facet_row_spacing=0.015,
            category_orders={"determinand": list(labels_mapping.keys())},
            height=5000,
            width=1000,
            )

# customise hover label to just show the value
fig.update_traces(
    hovertemplate="<br>".join([
        "%{customdata[0]}",
    ])
)

# Show all traces on the hover label
fig.update_layout(hovermode="x")

# prevent the Y axis from being shared between the subplots
fig.update_yaxes(matches=None)

# remove the "Determinand=" Facet Labels
fig.for_each_annotation(lambda a: a.update(text=labels_mapping[a.text.split("=")[-1]]))

fig.for_each_annotation(lambda a: a.update(textangle=0, x=0.4, y = a['y']+0.02))

fig.for_each_yaxis(lambda a: a.update(title=""))
fig.update_xaxes(title="", showticklabels=True)

# Dark mode doesn't show well in sharepoint
fig.update_layout({'plot_bgcolor': "#2a2a2a",
                    'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='DimGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='DimGrey')
                           

fig.show()


In [ ]:
js = '''document.body.style.backgroundColor = "#191919"; '''
fig.write_html("./docs/index.html", post_script=[js])

In [ ]:
lab_df
new_df = lab_df[lab_df["determinand"] == "Ammonia"].copy()

# pivot_df = lab_df.pivot(index = ['timestamp'], columns=["tank",'determinand'], values="value")
# pivot_df
new_df